# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

print("Sdk version is:", azureml.core.VERSION)

Sdk version is: 1.22.0


##### Set up workspace and experiment

* Set up compute cluster

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-exp'
project_folder = "./automl-project"

exp=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

cluster_name= "AML-Cluster"

#Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_v2',
                                                           min_nodes=1, max_nodes=5)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
compute_target.get_status()

Creating
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Breast Cancer is the most common type of cancer women experience worldwide, it is usually first observed in lumps or masses that develop in the breast, while some of these lumps or masses may not be cancerous and may be due to other hormonal complications, some of these lumps turn out to be cancerous. This is why this dataset generated focuses mainly on the side, texture and features of the lumps to determine if they will be cancerous or not.

In this data, I have 6 columns:
* The mean radius
* mean texture
* mean perimeter
* mean area
* mean smoothness
* diagnosis

The Diagnosis feature is the labeled data for our classification experiment that gives the input for women whose lumps are diagnosed as cancerous or not.
The column is already encoded where 1 means a cancerous lump and 0 means a non cancerous lump.

For this project, I'll be using the breast cancer dataset that contains features centered on the size and area of the tumor to determine if it's cancerous or not, or otherwise: malignant or benign.

This task is going to be a classification task which decides only two variables; cancerous or not.

In [4]:
#load the data
found = False
key = "breast-cancer-data"
description_text = "breast cancer prediction for third Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        data = 'https://raw.githubusercontent.com/Ayoyinka-Sofuwa/Capstone-project/main/Breast_cancer_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()

df.describe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In my automl settings, I set the experiment to time out at 30 minutes, run 4 experiments at a time and the primary metric to be highlighted will be the accuracy of each model generated.

It is a classification experiment and my target column to be predicted is the diagnosis column. And I configured the automated ML experiment to be Onnx compatible for deployment.

In [5]:
#Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                   "max_concurrent_iterations": 4,
                   "primary_metric" : 'accuracy'}

#Put your automl config here
automl_config = AutoMLConfig(task = "classification",
                             training_data=dataset,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'off',
                             enable_onnx_compatible_models=True,
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [6]:
#Submit your experiment
automl_run = exp.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
automl_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

*******************************************************************************

{'runId': 'AutoML_f522ea1c-2bb4-412c-b0e2-702869f6a55f',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-03-09T20:05:52.602983Z',
 'endTimeUtc': '2021-03-09T20:25:13.660813Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '10',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-exp","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-140119","workspace_name":"quick-starts-ws-140119","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":10,"y_min":null,"y_max":null,"num_classes":2,"feat

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_model, fitted_model = automl_run.get_output()

print(best_model)

Run(Experiment: automl-exp,
Id: AutoML_f522ea1c-2bb4-412c-b0e2-702869f6a55f_38,
Type: None,
Status: Completed)


In [10]:
fitted_model

Pipeline(memory=None,
         steps=[('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=None,
                                               estimators=[('24',
                                                            Pipeline(memory=None,
                                                                     steps=[('MaxAbsScaler',
                                                                             MaxAbsScaler(copy=True)),
                                                                            ('LightGBMClassifier',
                                                                             LightGBMClassifier(boosting_type='goss',
                                                                                                class_weight=None,
                                                                                                colsample_bytree=0.8911111111111111,
                                            

In [11]:
best_model.get_metrics()

{'AUC_macro': 0.9817803508483784,
 'recall_score_weighted': 0.9384398496240601,
 'matthews_correlation': 0.8678266159542292,
 'weighted_accuracy': 0.9436687497172818,
 'AUC_micro': 0.9822800755805554,
 'norm_macro_recall': 0.8653370921297079,
 'recall_score_micro': 0.9384398496240601,
 'f1_score_macro': 0.9328360009934242,
 'f1_score_micro': 0.9384398496240601,
 'f1_score_weighted': 0.9382077470164585,
 'AUC_weighted': 0.9817803508483787,
 'average_precision_score_weighted': 0.9835508280236338,
 'accuracy': 0.9384398496240601,
 'log_loss': 0.18089927319883362,
 'average_precision_score_macro': 0.9820583997779491,
 'precision_score_macro': 0.9353620924840564,
 'average_precision_score_micro': 0.9830288961871847,
 'precision_score_micro': 0.9384398496240601,
 'precision_score_weighted': 0.9399361548388994,
 'recall_score_macro': 0.932668546064854,
 'balanced_accuracy': 0.932668546064854,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_f522ea1c-2bb4-412c-b0e2-702869f6a55f_3

In [12]:
metrics = best_model.get_metrics()
metrics['accuracy']

0.9384398496240601

In [13]:
#TODO: Save the best model
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_automl_model, onnx_model= automl_run.get_output(return_onnx_model=True)

In [14]:
onnx_path = "./best_automl_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, file_path=onnx_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
automl_model = automl_run.register_model(model_name='best_automl_model', description='The best automl experiment model')

In [21]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import Model
from azureml.automl.core.shared import constants
from azureml.core.webservice import Webservice

from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True)

model = Model(ws, 'best_automl_model')

In [41]:
best_model.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

In [47]:
name = 'best-model-deployment'

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)
service = Model.deploy(ws, name, [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [48]:
service.update(enable_app_insights=True)

In [50]:
print(service.state, service.get_keys, service.scoring_uri, service.swagger_uri, sep= '\n')

Healthy
<bound method Webservice.get_keys of AciWebservice(workspace=Workspace.create(name='quick-starts-ws-140119', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-140119'), name=best-model-deployment, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://e381f9b4-cd2a-41f3-b935-d2cd04fa48e9.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})>
http://e381f9b4-cd2a-41f3-b935-d2cd04fa48e9.southcentralus.azurecontainer.io/score
http://e381f9b4-cd2a-41f3-b935-d2cd04fa48e9.southcentralus.azurecontainer.io/swagger.json


In [57]:
%run endpoint.py

{"result": [0, 1]}


TODO: In the cell below, send a request to the web service you deployed to test it.

In [54]:
# load existing web service
from azureml.core.webservice import Webservice
service = Webservice(name='best-model-deployment', workspace=ws)

TODO: In the cell below, print the logs of the web service and delete the service

In [55]:
logs = service.get_logs()

In [56]:
for line in logs.split('\n'):
    print(line)

2021-03-09T22:12:26,063852800+00:00 - iot-server/run 
2021-03-09T22:12:26,073521400+00:00 - rsyslog/run 
2021-03-09T22:12:26,064836800+00:00 - gunicorn/run 
2021-03-09T22:12:26,096194900+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_6f3791fe7434448b4ebe2b0fd691d644/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6f3791fe7434448b4ebe2b0fd691d644/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6f3791fe7434448b4ebe2b0fd691d644/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6f3791fe7434448b4ebe2b0fd691d644/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_6f3791fe7434448b4ebe2b0fd691d644/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [58]:
compute_target.delete()